In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np

import matplotlib.pyplot as plt

# Specify that we want our tensors on the CPU/GPU and in float32
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float32

In [2]:
train_dataset = datasets.mnist.MNIST('./data', transform=transforms.ToTensor(), train=True)
val_dataset = datasets.mnist.MNIST('./data', transform=transforms.ToTensor(), train=False)

In [3]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [4]:
class MNIST_AE(nn.Module):
    def __init__(self, width, height):
        super(MNIST_AE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride = 1), # 26x26x16
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 13x13x16
            nn.Conv2d(16, 8, kernel_size=3, stride = 1), # 11x11x8
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2) # 5x5x8
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(5*5*8, 3)

        self.decoder = nn.Sequential(
            nn.Linear(3, 5*5*8),
            nn.ReLU(),
            nn.Unflatten(1, (8, 5, 5)),
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2), # 11x11x16
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, kernel_size=3, stride=3, padding=3, output_padding=1), # 28x28x1
            nn.Sigmoid()
        )
    
    def forward(self, image_tensor):
        encoded = self.encoder(image_tensor)
        flattened = self.flatten(encoded)
        latent = self.fc(flattened)
        decoded = self.decoder(latent)
        return decoded, latent

In [5]:
mnist_ae = MNIST_AE(28, 28).to(device)
decoded, latent = mnist_ae.forward(train_dataset[0][0].to(device).reshape(1, 1, 28, 28))
print(f"Decoded shape: {decoded.shape}")
print(f"Latent shape: {latent.shape}")

Decoded shape: torch.Size([1, 1, 28, 28])
Latent shape: torch.Size([1, 3])


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [6]:
def reconstruction_loss(reconstructed, original):
    return torch.mean((original - reconstructed)**2)

In [7]:
reconstruction_loss(train_dataset[0][0].to(device).reshape(1, 1, 28, 28), decoded)

tensor(0.2311, grad_fn=<MeanBackward0>)

In [8]:
from tqdm.auto import trange
import torch.optim as optim
from copy import deepcopy
from torch.optim import lr_scheduler

def train_model(model, 
                train_dataset, 
                val_dataset,
                objective,
                regularizer=None,
                num_epochs=100, 
                lr=0.1,
                momentum=0.9,
                lr_step_size=25,
                lr_gamma=0.9):
    # progress bars
    pbar = trange(num_epochs)
    pbar.set_description("---")
    inner_pbar = trange(len(train_dataset))
    inner_pbar.set_description("Batch")

    # data loaders for train and validation
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32)
    dataloaders = dict(train=train_dataloader, val=val_dataloader)

    # use standard SGD with a decaying learning rate
    optimizer = optim.SGD(model.parameters(), 
                          lr=lr, 
                          momentum=momentum)
    scheduler = lr_scheduler.StepLR(optimizer, 
                                    step_size=lr_step_size, 
                                    gamma=lr_gamma)
    
    # Keep track of the best model
    best_model_wts = deepcopy(model.state_dict())
    best_loss = 1e8

    # Track the train and validation loss
    train_losses = []
    val_losses = []
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            # set model to train/validation as appropriate
            if phase == 'train':
                model.train()
                inner_pbar.reset()
            else:
                model.eval()
            
            # track the running loss over batches
            running_loss = 0
            running_size = 0
            for _, (data, _) in enumerate(dataloaders[phase]):
                if phase == "train":
                    with torch.set_grad_enabled(True):
                        optimizer.zero_grad()
                        # compute the model output and loss
                        output_t, _ = model(data)
                        loss_t = objective(output_t, data)
                        # only add the regularizer in the training phase
                        if regularizer is not None:
                            loss_t += regularizer(model)

                        # take the gradient and perform an sgd step
                        loss_t.backward()
                        optimizer.step()
                    inner_pbar.update(1)
                else:
                    # just compute the loss in validation
                    output_t, _ = model(data)
                    loss_t = objective(output_t, data)

                assert torch.isfinite(loss_t)
                running_loss += loss_t.item()
                running_size += 1
            
            # compute the train/validation loss and update the best
            # model parameters if this is the lowest validation loss yet
            running_loss /= running_size
            if phase == "train":
                train_losses.append(running_loss)
            else:
                val_losses.append(running_loss)
                if running_loss < best_loss:
                    best_loss = running_loss
                    best_model_wts = deepcopy(model.state_dict())

        # Update the learning rate
        scheduler.step()

        # Update the progress bar
        pbar.set_description("Epoch {:03} Train {:.4f} Val {:.4f}"\
                             .format(epoch, train_losses[-1], val_losses[-1]))
        pbar.update(1)

    # load best model weights
    model.load_state_dict(best_model_wts)

    return torch.tensor(train_losses), torch.tensor(val_losses)

/Users/JosephYu/opt/anaconda3/envs/nbd_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Construct an GLM with random initial weights.
torch.manual_seed(0)
mnist_ae = MNIST_AE(28, 28).to(device)

# Fit the GLM
print("Training Autoencoder...")
train_losses, val_losses = \
    train_model(mnist_ae, 
                train_dataset, 
                val_dataset, 
                reconstruction_loss,
                num_epochs=10
                )

In [2]:
import collections

class Solution:
    def wallsAndGates(self, rooms: list) -> None:
        """
        Do not return anything, modify rooms in-place instead.
        """
        self.inf = 2**31 - 1
        queue = collections.deque()
        seen = set()

        N = len(rooms)
        M = len(rooms[0])

        dist_to_gate = [[self.inf for _ in range(M)] for _ in range(N)]

        for r in range(N):
            for c in range(M):
                if rooms[r][c] == 0:
                    queue.append((r,c, 0))
                    seen.add((r,c))
                elif rooms[r][c] == -1:
                    dist_to_gate[r][c] = -1
        
        while queue:
            r, c, dist = queue.popleft()

            dist_to_gate[r][c] = dist
        
            for n_r, n_c in {(r+1, c), (r-1, c), (r, c+1), (r, c-1)}:
                if n_r >= 0 and n_c >= 0 and n_r < N and n_c < M and (n_r, n_c) not in seen and rooms[n_r][n_c] == self.inf:
                    seen.add((n_r, n_c))
                    queue.append((n_r, n_c, dist+1))
        
        return dist_to_gate

In [3]:
rooms = [[2147483647,-1,0,2147483647],[2147483647,2147483647,2147483647,-1],[2147483647,-1,2147483647,-1],[0,-1,2147483647,2147483647]]

sol = Solution()
sol.wallsAndGates(rooms)

[[3, -1, 0, 1], [2, 2, 1, -1], [1, -1, 2, -1], [0, -1, 3, 4]]


[[3, -1, 0, 1], [2, 2, 1, -1], [1, -1, 2, -1], [0, -1, 3, 4]]